In [ ]:
#MODEL_TYPE = "LSTM" # Choose between LSTM, DeepAR and MOIRAI

In [2]:
%cd /content

/content


In [3]:
# Initialize Directories and Configurations
print("LSTM model type stuff")

print("DeepAR model type stuff")
# install with support for torch models
!pip install "gluonts[torch]"
# install with support for mxnet models
!pip install "gluonts[mxnet]"

print("MOIRAI model type stuff")
!git clone https://github.com/taschoebli/uni2ts.git
%cd uni2ts
!pip install -e '.[notebook]'
#!pip install uni2ts
!touch .env


LSTM model type stuff
DeepAR model type stuff
MOIRAI model type stuff
fatal: destination path 'uni2ts' already exists and is not an empty directory.
/content/uni2ts
Obtaining file:///content/uni2ts
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 13.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [1]:
#General imports
print("General imports loaded")
import os
import pandas as pd
import numpy as np
import torch
import math
from torch import nn
from sklearn.metrics import mean_absolute_error, r2_score
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

#if MODEL_TYPE == "LSTM":
print("LSTM specific imports loaded")
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#elif MODEL_TYPE == "DeepAR":
print("DeepAR specific imports loaded")
from gluonts.torch import DeepAREstimator
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

#elif MODEL_TYPE == "MOIRAI":
print("MOIRAI specific imports loaded")
from huggingface_hub import hf_hub_download
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.model.moirai_moe import MoiraiMoEForecast, MoiraiMoEModule
from gluonts.dataset.pandas import PandasDataset
from gluonts.dataset.split import split

#else:
  #print("Invalid model type")


General imports loaded
LSTM specific imports loaded
DeepAR specific imports loaded
MOIRAI specific imports loaded


In [39]:
# Functions, Classes and other Helpers

# General ----------------------------------------------------------------------

# Load data from csv
def load_data(filepath):
    data = pd.read_csv(filepath, index_col=0, parse_dates=True)
    return data

# Filename extractor
def file_name_no_extension(filepath):
  # Extract the filename without the folder path
  filename_with_extension = os.path.basename(filepath)

  # Remove the `.csv` extension
  filename_without_extension = os.path.splitext(filename_with_extension)[0]

  return filename_without_extension


# Day extractor
def get_values_by_day(df_internal, offset=0):
  # Ensure index is datetime
  if not isinstance(df_internal.index, pd.DatetimeIndex):
      df_internal.index = pd.to_datetime(df_internal.index)

  # Apply offset by removing the last 'offset' rows
  if offset > 0:
      df_internal = df_internal.iloc[:-offset]

  # Filter timestamps to only include those at exactly 08:00
  df_filtered = df_internal[df_internal.index.hour == 8]

  # Group by date and store in a dictionary
  #result = {}
  #for date, group in df_filtered.groupby(df_filtered.index.date):
      #result[str(date)] = group.index.tolist()
  # Extract timestamps as strings instead of lists
  result = {group.index[0].strftime('%Y-%m-%d %H:%M:%S') for date, group in df_filtered.groupby(df_filtered.index.date)}

  return result


# Calculate SMAPE metric
def calc_smape(y_true, y_pred):
  """
  Calculate the Symmetric Mean Absolute Percentage Error (SMAPE).
  y_true: array-like, actual values
  y_pred: array-like, predicted values
  """
  epsilon = 1e-10  # Small constant to avoid division by zero
  denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
  smape_values = np.abs(y_true - y_pred) / (denominator + epsilon)
  smape_values = np.nan_to_num(smape_values, nan=0.0, posinf=0.0, neginf=0.0)  # Handle division by zero

  return np.mean(smape_values) * 100  # Convert to percentage

def optimize_memory():
  # Memory optimization
  # Set CUDA memory management configuration to avoid fragmentation
  # https://pytorch.org/docs/stable/notes/cuda.html#using-custom-memory-allocators-for-cuda
  os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
  # Check if CUDA is available
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  print(f"Using device: {device}")
  torch.cuda.empty_cache()
  os.environ["PYTORCH_CUDA_ALLOC_CONF"]


# LSTM specific ----------------------------------------------------------------

# Create sequences for time-series prediction
def create_sequences(features, target, context_length, prediction_length):
    X_seq, y_seq = [], []
    for i in range(len(features) - context_length - prediction_length + 1):
        X_seq.append(features[i:i+context_length])  # Past `context_length` values
        y_seq.append(target[i+context_length:i+context_length+prediction_length])  # Next `prediction_length` values
    return np.array(X_seq), np.array(y_seq)

# Define the LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])  # Use the last hidden state
        return out

def train_LSTM_model(model, train_loader, criterion, optimizer, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader):.4f}')
    return epoch_loss/len(train_loader)

def evaluate_LSTM_model(model, test_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            predictions.extend(outputs.tolist())
            actuals.extend(y_batch.tolist())
    # Feature added here!!!
    # Truncate negative predictions to 0
    predictions = np.maximum(predictions, 0)
    return np.array(predictions), np.array(actuals)

def get_index_and_timestamp_for_validation(hour, original_df, train_size):
    """
    Finds the first available testdata tensor index that matches the given hour.

    Parameters:
    - hour (int): The desired hour (0-23).
    - original_df (pd.DataFrame): The original dataset with a datetime index.
    - train_size (int): The number of samples in the training dataset + Context Length.

    Returns:
    - test_index (int): The index relative to testdata tensor.
    - timestamp (str): The corresponding datetime string.
    """
    # Ensure datetime format
    original_df.index = pd.to_datetime(original_df.index)

    # Extract test dataset portion
    test_df = original_df.iloc[train_size:]  # This contains only the test set

    # Find the first row where the hour matches
    for i, timestamp in enumerate(test_df.index):
        if timestamp.hour == hour:
            test_index = i  # Relative index in testdata tensor
            return test_index, timestamp.strftime('%Y-%m-%d %H:%M:%S')

    raise ValueError(f"No test data found at hour {hour}:00.")


def truncate_tensor_with_interval(tensor, start_index, offset=1):
    """
    Truncates the given tensor along the first dimension, starting at start_index,
    and selects every `offset`-th row.

    Parameters:
    - tensor (torch.Tensor): The input tensor (shape: [X, Y]).
    - start_index (int): The base index from which truncation starts.
    - offset (int): Step interval to select elements (e.g., every 4th, 12th, etc.).

    Returns:
    - truncated_tensor (torch.Tensor): The truncated tensor with selected intervals.
    """
    if start_index >= tensor.shape[0]:
        raise ValueError("Start index exceeds tensor dimensions.")

    # Make sure that below 24 hours, offset is set to one day!
    if offset < 24:
      offset = 24

    return tensor[start_index::offset]  # Truncate and select every `offset` step

def data_validation_LSTM(df, actuals, X_train, CONTEXT_LENGTH, PREDICTION_LENGTH, hour=8):
    """
    Validates the actual LSTM predictions against the true values in the dataset.

    Parameters:
    - df (pd.DataFrame): The original dataset with a datetime index.
    - actuals (torch.Tensor): The model's predicted values (shape: [X, Y]).
    - X_train (numpy array or tensor): The training dataset.
    - CONTEXT_LENGTH (int): The number of past timesteps used for forecasting.
    - PREDICTION_LENGTH (int): The number of future timesteps predicted.
    - hour (int): The desired hour (default: 8 AM).


    """

    # Step 1: Get the test index and timestamp
    INDEX, timestamp = get_index_and_timestamp_for_validation(hour, df, train_size=len(X_train) + CONTEXT_LENGTH)

    # Step 2: Extract predictions using the computed index
    actuals_daily = truncate_tensor_with_interval(actuals, INDEX, offset=PREDICTION_LENGTH)

    # Step 3: Retrieve true values from `df['dc_power']`
    start_index = len(X_train) + CONTEXT_LENGTH + INDEX
    true_values = df['dc_power'].iloc[start_index:start_index + PREDICTION_LENGTH].values  # Convert to NumPy array

    # Step 4: Format values for comparison
    formatted_actuals = np.round(actuals_daily[0], 1)  # Convert tensor to NumPy & round
    formatted_true_values = np.round(true_values, 1)  # Round true values for comparison

    # Step 5: Print Debugging Information
    print("🔍 Validation at timestamp:", timestamp)
    print("Predicted Values (Rounded):")
    print(f"{formatted_actuals}")
    print("True Values (Rounded):")
    print(f"{formatted_true_values}")

    # Step 6: Perform Validation Check
    validation_passed = np.array_equal(formatted_actuals, formatted_true_values)

    print("\n✅ Validation Passed" if validation_passed else "\n❌ Validation Failed")

from datetime import datetime

def days_between_dates(date_str1: str, date_str2: str, date_format: str = "%Y-%m-%d %H:%M:%S") -> int:
    """
    Returns the number of days between two datetime strings.

    :param date_str1: First date string
    :param date_str2: Second date string
    :param date_format: Format of the date strings (default: "%Y-%m-%d %H:%M:%S")
    :return: Number of days between the two dates (absolute value)
    """
    date1 = datetime.strptime(date_str1, date_format)
    date2 = datetime.strptime(date_str2, date_format)
    return abs((date2 - date1).days)

In [46]:
# Definitions and Constants
FILEPATH = '/content/sample_data/230401_250108_PT1H_Solcast_reduced_features.csv'
#FILEPATH = '/content/sample_data/200101_220423_PT1H_Unisolar.csv'
PREDICTION_LENGTH_LIST = [4, 12, 24, 48, 96, 336] # Use the past 4, 12, 24, 48, 96, 336 time steps for prediction (4 hours to 14 days)
CONTEXT_LENGTH_FOLD_LIST = [1, 2, 4, 8] # Use 1, 2, 4 and 8 fold PREDICTION_LENGTH
BEGINNING_TIMESTAMP = "2024-08-31 08:00:00"
START_HOUR = 8 # Use to start plots at this hour
TRAIN_DATA_LAST_INDEX = 8784 # Corresponds to '2024-04-01 00:00:00', used for LSTM and DeepAR in 230401_250108_PT1H_Solcast_reduced_features.csv
#TRAIN_DATA_LAST_INDEX = 8785 # Corresponds to '2021-01-01 00:00:00', used for LSTM and DeepAR in 200101_220423_PT1H_Unisolar.csv

print("LSTM model definitions and constants loaded")
LSTM_HIDDEN_SIZE = 64  # Number of hidden units -> # of neurons in the LSTM's hidden layer
LSTM_BATCH_SIZE = 64  # Batch size -> # of samples processed in parallel
LSTM_NUM_LAYERS = 2  # Number of LSTM layers -> # of stacked LSTM layers
LSTM_LR = 1e-3
LSTM_NUM_EPOCHS = 25

print("DeepAR model type stuff")
DeepAR_HIDDEN_SIZE = 64 # Number of RNN cells for each layer (default: 40)
DeepAR_NUM_LAYERS = 2 # Number of RNN layers (default: 2)
DeepAR_LR = 1e-3
DeepAR_NUM_EPOCHS = 25

print("MOIRAI model type stuff")
MOIRAI_PATCH_SIZE = 32 # patch size, Number of samples for each layer or sequence, 32 or 64 recommended for hourly
MOIRAI_BATCH_SIZE = 16 # batch size, samples processed in parallel
MOIRAI_MODEL = "moirai-1.1-R"  # model name: choose from {'moirai-1.1-R', 'moirai-moe-1.0-R'}
MOIRAI_SIZE = "base"  # model size: choose from {'small', 'base', 'large'}
MOIRAI_MODEL_STR = f"Salesforce/{MOIRAI_MODEL}-{MOIRAI_SIZE}"
MOIRAI_MODEL_PATH_SAFE = MOIRAI_MODEL_STR.replace("Salesforce/", "").replace("/", "-").replace(".", "-")

optimize_memory()

LSTM model definitions and constants loaded
DeepAR model type stuff
MOIRAI model type stuff
Using device: cpu


In [47]:
# Step 1: Load data from csv
df = load_data(FILEPATH)

In [48]:
## Step 2: Feature Scaling
print("LSTM feature scaling")
# Separate features and target
LSTM_target_column = 'dc_power'
LSTM_features = [
    'air_temp', 'albedo', 'azimuth', 'clearsky_dhi', 'clearsky_dni', 'clearsky_ghi', 'clearsky_gti',
    'cloud_opacity', 'dewpoint_temp', 'dhi', 'dni', 'ghi', 'gti', 'precipitable_water',
    'precipitation_rate', 'relative_humidity', 'surface_pressure', 'snow_depth',
    'snow_water_equivalent', 'snow_soiling_rooftop', 'snow_soiling_ground',
    'wind_direction_100m', 'wind_direction_10m', 'wind_speed_100m', 'wind_speed_10m', 'zenith'
]

# For Solcast
LSTM_features = [
    'air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith'
]
"""
# For Unisolar
LSTM_features = [
    'CloudOpacity', 'Ghi', 'ApparentTemperature', 'AirTemperature', 'DewPointTemperature', 'RelativeHumidity', 'WindSpeed', 'WindDirection'
]
"""
X = df[LSTM_features]
y = df[LSTM_target_column]

# Standardize the features
scaler = StandardScaler()
#scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert target to a numpy array
y = y.to_numpy()

print("DeepAR OR MOIRAI feature scaling")
# https://ts.gluon.ai/dev/api/gluonts/gluonts.dataset.pandas.html
# Convert into GluonTS dataset with features

# For Solcast
dataset = PandasDataset(df, target="dc_power",
                      past_feat_dynamic_real=['dc_power', 'air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith'],
                      feat_dynamic_real=['air_temp', 'azimuth', 'cloud_opacity', 'dhi', 'dni', 'ghi', 'zenith'])
"""
# For Unisolar
dataset = PandasDataset(df, target="dc_power",
                      past_feat_dynamic_real=['dc_power', 'CloudOpacity', 'Ghi', 'ApparentTemperature', 'AirTemperature', 'DewPointTemperature', 'RelativeHumidity', 'WindSpeed', 'WindDirection'],
                      feat_dynamic_real=['CloudOpacity', 'Ghi', 'ApparentTemperature', 'AirTemperature', 'DewPointTemperature', 'RelativeHumidity', 'WindSpeed', 'WindDirection'])
"""

LSTM feature scaling
DeepAR OR MOIRAI feature scaling


'\n# For Unisolar\ndataset = PandasDataset(df, target="dc_power",\n                      past_feat_dynamic_real=[\'dc_power\', \'CloudOpacity\', \'Ghi\', \'ApparentTemperature\', \'AirTemperature\', \'DewPointTemperature\', \'RelativeHumidity\', \'WindSpeed\', \'WindDirection\'],\n                      feat_dynamic_real=[\'CloudOpacity\', \'Ghi\', \'ApparentTemperature\', \'AirTemperature\', \'DewPointTemperature\', \'RelativeHumidity\', \'WindSpeed\', \'WindDirection\'])\n'

In [ ]:
# Main block - run for several iterations

for prediction_length in sorted(PREDICTION_LENGTH_LIST):
  for context_length_fold in sorted(CONTEXT_LENGTH_FOLD_LIST):
    PREDICTION_LENGTH = prediction_length
    CONTEXT_LENGTH = context_length_fold * PREDICTION_LENGTH

    print("LSTM model - Main block")
    # Step 3: Create sequences
    X_seq, y_seq = create_sequences(X_scaled, y, CONTEXT_LENGTH, PREDICTION_LENGTH)

    # Step 4: Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, train_size=TRAIN_DATA_LAST_INDEX, random_state=None, shuffle=False)
    # Convert data to PyTorch tensors
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_test = torch.tensor(X_test, dtype=torch.float32)
    y_test = torch.tensor(y_test, dtype=torch.float32)

    # Step 5: Create DataLoader for batch processing
    train_dataset = TensorDataset(X_train, y_train)
    test_dataset = TensorDataset(X_test, y_test)
    train_loader = DataLoader(train_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=LSTM_BATCH_SIZE, shuffle=False)

    # Step 6: Create the model
    input_size = X_train.shape[2]  # Number of features -> 7 eg. 'air_temp', 'albedo', etc.
    model = LSTMModel(input_size, LSTM_HIDDEN_SIZE, LSTM_NUM_LAYERS, output_size=PREDICTION_LENGTH)

    # Step 7: Define the loss function and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), LSTM_LR)

    # Step 8: Train the model
    LOSS = train_LSTM_model(model, train_loader, criterion, optimizer, LSTM_NUM_EPOCHS)

    # Step 9: Evaluate the model
    LSTM_predictions, LSTM_actuals = evaluate_LSTM_model(model, test_loader)
    data_validation_LSTM(df, LSTM_actuals, X_train, CONTEXT_LENGTH, PREDICTION_LENGTH, hour=START_HOUR)
    INDEX, TIMESTAMP = get_index_and_timestamp_for_validation(START_HOUR, df, train_size=len(X_train) + CONTEXT_LENGTH)
    #TIMESTAMP_DATE = TIMESTAMP.split(" ")[0]
    #LSTM_actuals_daily = truncate_tensor_with_interval(LSTM_actuals, INDEX, offset=PREDICTION_LENGTH)
    #LSTM_predictions_daily = truncate_tensor_with_interval(LSTM_predictions, INDEX, offset=PREDICTION_LENGTH)

    # To allign with MOIRAI and DeepAR
    LSTM_PREDICTIONS_INDEX_SHIFTED = days_between_dates(TIMESTAMP, BEGINNING_TIMESTAMP) * 24

    # Step 10: Calc metrics

    LSTM_rmse = np.sqrt(np.mean((LSTM_predictions - LSTM_actuals)**2))
    LSTM_mae = mean_absolute_error(LSTM_actuals, LSTM_predictions)
    LSTM_r2 = r2_score(LSTM_actuals, LSTM_predictions)
    LSTM_smape = calc_smape(LSTM_actuals, LSTM_predictions)
    print(f"PL={PREDICTION_LENGTH}, CL={CONTEXT_LENGTH}, Model=LSTM")
    #print(f"Length of: RMSE={len(LSTM_rmse_list)}, MAE={len(LSTM_mae_list)}, SMAPE={len(LSTM_smape_list)}, R2={len(LSTM_r2_list)}")
    #print(f"Mean of: RMSE={np.mean(LSTM_rmse_list):.4f}, MAE={np.mean(LSTM_mae_list):.4f}, SMAPE={np.mean(LSTM_smape_list):.2f}%, R^2={np.mean(LSTM_r2_list):.4f}")
    #LSTM_RMSE_MED = np.median(LSTM_rmse_list)
    #LSTM_MAE_MED = np.median(LSTM_mae_list)
    #LSTM_SMAPE_MED = np.median(LSTM_smape_list)
    #LSTM_R2_MED = np.median(LSTM_r2_list)
    #print(f"Median of: RMSE={LSTM_RMSE_MED:.4f}, MAE={LSTM_MAE_MED:.4f}, SMAPE={LSTM_SMAPE_MED:.2f}%, R^2={LSTM_R2_MED:.4f}")
    print("")


    print("DeepAR model type stuff")

    # Step 3: Split data into training and testing sets
    TESTDATA_LENGTH = len(df) - TRAIN_DATA_LAST_INDEX

    # Step 4: Create test sequences for time-series prediction
    training_data, test_gen = split(dataset, date=pd.Period(BEGINNING_TIMESTAMP, freq="h"))
    test_data = test_gen.generate_instances(prediction_length=PREDICTION_LENGTH, windows=1)

    # Step5: Define and Train the model
    model = DeepAREstimator(
        prediction_length=PREDICTION_LENGTH, context_length=CONTEXT_LENGTH, freq=dataset.freq,
        trainer_kwargs={"max_epochs": DeepAR_NUM_EPOCHS},
        num_feat_dynamic_real=dataset.num_feat_dynamic_real,
        num_layers=DeepAR_NUM_LAYERS,
        hidden_size=DeepAR_HIDDEN_SIZE,
        lr=DeepAR_LR,
    ).train(training_data)

    # Step 6: Get probabilistic predictions
    forecasts = list(model.predict(test_data.input))

    # Step 7: Get point predictions
    # Get predictions from forecasts
    predictions = forecasts[0].mean_ts

    # Feature added here!!!
    # Truncate negative predictions to 0
    predictions = np.maximum(predictions, 0)

    # Get actuals for metric calculations later
    actuals = df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]["dc_power"]

    # Change type
    DeepAR_predictions = predictions.to_numpy()
    DeepAR_actuals = actuals.to_numpy()

    # Step 8: Do evaluations
    DeepAR_rmse = np.sqrt(np.mean((DeepAR_predictions - DeepAR_actuals)**2))
    DeepAR_mae = mean_absolute_error(DeepAR_actuals, DeepAR_predictions)
    DeepAR_r2 = r2_score(DeepAR_actuals, DeepAR_predictions)
    DeepAR_smape = calc_smape(DeepAR_actuals, DeepAR_predictions)

    print(f"PL={PREDICTION_LENGTH}, CL={CONTEXT_LENGTH}, Model=DeepAR")
    #print(f"Length of: RMSE={len(DeepAR_rmse_list)}, MAE={len(DeepAR_mae_list)}, SMAPE={len(DeepAR_smape_list)}, R2={len(DeepAR_r2_list)}")
    #print(f"Mean of: RMSE={np.mean(DeepAR_rmse_list):.4f}, MAE={np.mean(DeepAR_mae_list):.4f}, SMAPE={np.mean(DeepAR_smape_list):.2f}%, R^2={np.mean(DeepAR_r2_list):.4f}")
    #DeepAR_RMSE_MED = np.median(DeepAR_rmse_list)
    #DeepAR_MAE_MED = np.median(DeepAR_mae_list)
    #DeepAR_SMAPE_MED = np.median(DeepAR_smape_list)
    #DeepAR_R2_MED = np.median(DeepAR_r2_list)
    #print(f"Median of: RMSE={DeepAR_RMSE_MED:.4f}, MAE={DeepAR_MAE_MED:.4f}, SMAPE={DeepAR_SMAPE_MED:.2f}%, R^2={DeepAR_R2_MED:.4f}")
    #print("")

    # Prepare for Plot
    # Get prediction intervals
    DeepAR_lower_50 = forecasts[0].quantile(0.25)  # 25th percentile (lower bound)
    DeepAR_upper_50 = forecasts[0].quantile(0.75)  # 75th percentile (upper bound)
    DeepAR_lower_90 = forecasts[0].quantile(0.05)  # 5th percentile (lower bound)
    DeepAR_upper_90 = forecasts[0].quantile(0.95)  # 95th percentile (upper bound)


    print("MOIRAI model type stuff")

    # Step 2.1: Prepare pre-trained model by downloading model weights from huggingface hub
    if "moirai-moe" in MOIRAI_MODEL_STR:
      model = MoiraiMoEForecast(
        module=MoiraiMoEModule.from_pretrained(MOIRAI_MODEL_STR),
        prediction_length=PREDICTION_LENGTH,
        context_length=CONTEXT_LENGTH,
        patch_size=MOIRAI_PATCH_SIZE,
        num_samples=100,
        target_dim=1,
        feat_dynamic_real_dim=dataset.num_feat_dynamic_real,
        past_feat_dynamic_real_dim=dataset.num_past_feat_dynamic_real,
      )
      #print("MOE used")
    else:
      model = MoiraiForecast(
        module=MoiraiModule.from_pretrained(MOIRAI_MODEL_STR),
        prediction_length=PREDICTION_LENGTH,
        context_length=CONTEXT_LENGTH,
        patch_size=MOIRAI_PATCH_SIZE,
        num_samples=100,
        target_dim=1,
        feat_dynamic_real_dim=dataset.num_feat_dynamic_real,
        past_feat_dynamic_real_dim=dataset.num_past_feat_dynamic_real,
      )
      #print("No MOE used")

    '''

    # Step 2.2: Prepare fine-tuned model by loading model weights from ckpt file
    checkpoint_path = "/content/last.ckpt"

    # Load the model
    model = MoiraiForecast.load_from_checkpoint(checkpoint_path,
    prediction_length=PREDICTION_LENGTH,
    context_length=CONTEXT_LENGTH,
    patch_size=PATCH_SIZE,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=dataset.num_feat_dynamic_real,
    past_feat_dynamic_real_dim=dataset.num_past_feat_dynamic_real)
    '''

    # Step 6: Get probabilistic predictions
    predictor = model.create_predictor(batch_size=MOIRAI_BATCH_SIZE)
    forecasts = list(predictor.predict(test_data.input))

    # Step 7: Get point predictions
    # Get predictions from forecasts
    predictions = forecasts[0].mean_ts
    # Feature added here!!!
    # Truncate negative predictions to 0
    predictions = np.maximum(predictions, 0)

    # Get actuals for metric calculations later
    actuals = df.loc[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]["dc_power"]

    # Change type
    MOIRAI_predictions = predictions.to_numpy()
    MOIRAI_actuals = actuals.to_numpy()

    # Step 8: Do evaluations
    MOIRAI_rmse = np.sqrt(np.mean((MOIRAI_predictions - MOIRAI_actuals)**2))
    MOIRAI_mae = mean_absolute_error(MOIRAI_actuals, MOIRAI_predictions)
    MOIRAI_r2 = r2_score(MOIRAI_actuals, MOIRAI_predictions)
    MOIRAI_smape = calc_smape(MOIRAI_actuals, MOIRAI_predictions)

    print(f"PL={PREDICTION_LENGTH}, CL={CONTEXT_LENGTH}, Model={MOIRAI_MODEL_PATH_SAFE}")
    #print(f"Length of: RMSE={len(MOIRAI_rmse_list)}, MAE={len(MOIRAI_mae_list)}, SMAPE={len(MOIRAI_smape_list)}, R2={len(MOIRAI_r2_list)}")
    #print(f"Mean of: RMSE={np.mean(MOIRAI_rmse_list):.4f}, MAE={np.mean(MOIRAI_mae_list):.4f}, SMAPE={np.mean(MOIRAI_smape_list):.2f}%, R^2={np.mean(MOIRAI_r2_list):.4f}")
    #MOIRAI_RMSE_MED = np.median(MOIRAI_rmse_list)
    #MOIRAI_MAE_MED = np.median(MOIRAI_mae_list)
    #MOIRAI_SMAPE_MED = np.median(MOIRAI_smape_list)
    #MOIRAI_R2_MED = np.median(MOIRAI_r2_list)
    #print(f"Median of: RMSE={MOIRAI_RMSE_MED:.4f}, MAE={MOIRAI_MAE_MED:.4f}, SMAPE={MOIRAI_SMAPE_MED:.2f}%, R^2={MOIRAI_R2_MED:.4f}")
    print("")


    # Prepare for Plot
    # Get prediction intervals
    MOIRAI_lower_50 = forecasts[0].quantile(0.25)  # 25th percentile (lower bound)
    MOIRAI_upper_50 = forecasts[0].quantile(0.75)  # 75th percentile (upper bound)
    MOIRAI_lower_90 = forecasts[0].quantile(0.05)  # 5th percentile (lower bound)
    MOIRAI_upper_90 = forecasts[0].quantile(0.95)  # 95th percentile (upper bound)





    # Plot filtered data all togehter
    plt.plot(df[BEGINNING_TIMESTAMP:]["dc_power"].iloc[:PREDICTION_LENGTH], color="black", linestyle="--", label="True values")
    plt.plot(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index, MOIRAI_predictions, color="orange", label="MOIRAI Predictions")
    plt.plot(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index, DeepAR_predictions, color="blue", label="DeepAR Predictions")
    plt.plot(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index, LSTM_predictions[(LSTM_PREDICTIONS_INDEX_SHIFTED-1)+START_HOUR], color="green", label="LSTM Predictions")

    # Add 50% Prediction Interval (Shaded)
    plt.fill_between(
        df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index,
        MOIRAI_lower_50[len(MOIRAI_lower_50)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        MOIRAI_upper_50[len(MOIRAI_upper_50)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        color="orange", alpha=0.2, label="MOIRAI 50% PI"
    )

    # Add 90% Prediction Interval (Shaded)
    plt.fill_between(
        df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index,
        MOIRAI_lower_90[len(MOIRAI_lower_90)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        MOIRAI_upper_90[len(MOIRAI_upper_90)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        color="orange", alpha=0.075, label="MOIRAI 90% PI"
    )
    # Add 50% Prediction Interval (Shaded)
    plt.fill_between(
        df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index,
        DeepAR_lower_50[len(DeepAR_lower_50)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        DeepAR_upper_50[len(DeepAR_upper_50)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        color="blue", alpha=0.2, label="DeepAR 50% PI"
    )

    # Add 90% Prediction Interval (Shaded)
    plt.fill_between(
        df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH].index,
        DeepAR_lower_90[len(DeepAR_lower_90)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        DeepAR_upper_90[len(DeepAR_upper_90)-len(df[BEGINNING_TIMESTAMP:].iloc[:PREDICTION_LENGTH]):],
        color="blue", alpha=0.075, label="DeepAR 90% PI"
    )

    plt.legend(loc="upper right", fontsize="small")
    plt.ylabel("DC Power (kW)")
    plt.xlabel("Date")
    plt.title(f"LSTM, DeepAR & MOIRAI Base Forecast (starts {BEGINNING_TIMESTAMP})")
    plt.figtext(0.1, 0.9000, f"MOIRAI: RMSE={MOIRAI_rmse:.4f}, MAE={MOIRAI_mae:.4f}, SMAPE={MOIRAI_smape:.2f}%, R^2={MOIRAI_r2:.4f}", fontsize=6, color='black')
    plt.figtext(0.1, 0.8700, f"DeepAR: RMSE={DeepAR_rmse:.4f}, MAE={DeepAR_mae:.4f}, SMAPE={DeepAR_smape:.2f}%, R^2={DeepAR_r2:.4f}", fontsize=6, color='black')
    plt.figtext(0.1, 0.8400, f"LSTM: RMSE={LSTM_rmse:.4f}, MAE={LSTM_mae:.4f}, SMAPE={LSTM_smape:.2f}%, R^2={LSTM_r2:.4f}", fontsize=6, color='black')
    plt.figtext(0.1, 0.8100,f"LSTM&DeepAR&MOIRAI_Features={dataset.feat_dynamic_real}", fontsize=4, color='black')
    plt.figtext(0.1, 0.7800,f"DeepAR&MOIRAI_Past_Features={dataset.past_feat_dynamic_real}", fontsize=4, color='black')
    plt.figtext(0.1, 0.7500,f"LSTM: HIDDEN_SIZE={LSTM_HIDDEN_SIZE}, BATCH={LSTM_BATCH_SIZE}, LR={LSTM_LR}, Epochs={LSTM_NUM_EPOCHS}", fontsize=4, color='black')
    plt.figtext(0.1, 0.7200,f"MOIRAI: PATCH={MOIRAI_PATCH_SIZE}, BATCH={MOIRAI_BATCH_SIZE}, LR=n.A., Epochs=n.A.", fontsize=4, color='black')
    plt.figtext(0.1, 0.6900,f"DeepAR: HIDDEN_SIZE={DeepAR_HIDDEN_SIZE}, RNN_LAYERS={DeepAR_NUM_LAYERS}, LR={DeepAR_LR}, Epochs={DeepAR_NUM_EPOCHS}", fontsize=4, color='black')
    plt.figtext(0.1, 0.6600,f"Dataset={file_name_no_extension(FILEPATH)}", fontsize=4, color='black')
    plt.figtext(0.1, 0.6300,f"Pre_L={PREDICTION_LENGTH}, Con_L={CONTEXT_LENGTH}", fontsize=6, color='black')

    # Set y-axis limits
    plt.ylim(0, 12.5)

    # Set y-axis limits and x-axis formatting
    ax = plt.gca()
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    ax.xaxis.set_major_locator(mdates.DayLocator())
    plt.xticks(rotation=25)

    # Adjust layout and save plot
    plt.tight_layout()
    plt.savefig(f"PL={PREDICTION_LENGTH}-CL={CONTEXT_LENGTH}-{BEGINNING_TIMESTAMP}-LSTM_DeepAR_MOIRAIBase_Forecasts.pdf")
    #plt.show()



    # Reset plot before next round
    plt.figure().clear()
    plt.close()
    plt.cla()
    plt.clf()


LSTM model - Main block
Epoch 1/25, Loss: 5.5766
Epoch 2/25, Loss: 2.2802
Epoch 3/25, Loss: 1.8302
Epoch 4/25, Loss: 1.6729
Epoch 5/25, Loss: 1.5807
Epoch 6/25, Loss: 1.5283
Epoch 7/25, Loss: 1.5099
Epoch 8/25, Loss: 1.4783
Epoch 9/25, Loss: 1.4720
Epoch 10/25, Loss: 1.4459
Epoch 11/25, Loss: 1.4376
Epoch 12/25, Loss: 1.4069
Epoch 13/25, Loss: 1.3862
Epoch 14/25, Loss: 1.3529
Epoch 15/25, Loss: 1.3268
Epoch 16/25, Loss: 1.2980
Epoch 17/25, Loss: 1.2756
Epoch 18/25, Loss: 1.2529
Epoch 19/25, Loss: 1.2354
Epoch 20/25, Loss: 1.2182
Epoch 21/25, Loss: 1.2052
Epoch 22/25, Loss: 1.1898
Epoch 23/25, Loss: 1.1784
Epoch 24/25, Loss: 1.1633
Epoch 25/25, Loss: 1.1526
🔍 Validation at timestamp: 2024-04-01 08:00:00
Predicted Values (Rounded):
[0.  0.1 0.5 0.9]
True Values (Rounded):
[0.  0.1 0.5 0.9]

✅ Validation Passed
PL=4, CL=4, Model=LSTM

DeepAR model type stuff


INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: 
  | Name  | Type        | Params | Mode  | In sizes                                                       | Out sizes  
-----------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 64.5 K | train | [[1, 1], [1, 1], [1, 724, 12], [1, 724], [1, 724], [1, 4, 12]] | [1, 100, 4]
--------------------------------------------------------------------------------

Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached -3.20331 (best -3.20331), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached -3.20331 (best -3.20331), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached -4.64967 (best -4.64967), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -4.64967 (best -4.64967), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' reached -5.02163 (best -5.02163), saving model to '/content/lightning_logs/version_2/checkpoints/epoch=2-step=150.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 2, global s

PL=4, CL=4, Model=DeepAR
MOIRAI model type stuff
PL=4, CL=4, Model=moirai-1-1-R-base

LSTM model - Main block
Epoch 1/25, Loss: 5.4461
Epoch 2/25, Loss: 2.0760
Epoch 3/25, Loss: 1.7069
Epoch 4/25, Loss: 1.5613
Epoch 5/25, Loss: 1.4717
Epoch 6/25, Loss: 1.4510
Epoch 7/25, Loss: 1.4533
Epoch 8/25, Loss: 1.4441
Epoch 9/25, Loss: 1.4498
Epoch 10/25, Loss: 1.4287
Epoch 11/25, Loss: 1.4203
Epoch 12/25, Loss: 1.4016
Epoch 13/25, Loss: 1.3934
Epoch 14/25, Loss: 1.3713
Epoch 15/25, Loss: 1.3207
Epoch 16/25, Loss: 1.2768
Epoch 17/25, Loss: 1.2691
Epoch 18/25, Loss: 1.2365
Epoch 19/25, Loss: 1.2166
Epoch 20/25, Loss: 1.1731
Epoch 21/25, Loss: 1.1815
Epoch 22/25, Loss: 1.1397
Epoch 23/25, Loss: 1.1358
Epoch 24/25, Loss: 1.0981
Epoch 25/25, Loss: 1.0830


...

INFO: GPU available: False, used: False
INFO:lightning.pytorch.utilities.rank_zero:GPU available: False, used: False
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.11/dist-packages/lightning/pytorch/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
INFO: 
  | Name  | Type        | Params | Mode  | In sizes                                                        | Out sizes   
-------------------------------------------------------------------------------------------------------------------------------
0 | model | DeepARModel | 64.5 K | train | [[1, 1], [1, 1], [1, 744, 12], [1, 744], [1, 744], [1, 24, 12]] | [1, 100, 24]
--------------------------------------------------------------------------

🔍 Validation at timestamp: 2024-04-02 08:00:00
Predicted Values (Rounded):
[ 0.6  1.   2.8  3.6  3.1  3.7 10.  10.  10.   8.9  4.3  1.1  0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]
True Values (Rounded):
[ 0.6  1.   2.8  3.6  3.1  3.7 10.  10.  10.   8.9  4.3  1.1  0.   0.
  0.   0.   0.   0.   0.   0.   0.   0.   0.   0. ]

✅ Validation Passed
PL=24, CL=24, Model=LSTM

DeepAR model type stuff


Training: |          | 0/? [00:00<?, ?it/s]

INFO: Epoch 0, global step 50: 'train_loss' reached 0.57334 (best 0.57334), saving model to '/content/lightning_logs/version_10/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 0, global step 50: 'train_loss' reached 0.57334 (best 0.57334), saving model to '/content/lightning_logs/version_10/checkpoints/epoch=0-step=50.ckpt' as top 1
INFO: Epoch 1, global step 100: 'train_loss' reached -0.75290 (best -0.75290), saving model to '/content/lightning_logs/version_10/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 1, global step 100: 'train_loss' reached -0.75290 (best -0.75290), saving model to '/content/lightning_logs/version_10/checkpoints/epoch=1-step=100.ckpt' as top 1
INFO: Epoch 2, global step 150: 'train_loss' was not in top 1
INFO:lightning.pytorch.utilities.rank_zero:Epoch 2, global step 150: 'train_loss' was not in top 1
INFO: Epoch 3, global step 200: 'train_loss' reached -0.97342 (best -0.97

PL=24, CL=24, Model=DeepAR
MOIRAI model type stuff
PL=24, CL=24, Model=moirai-1-1-R-base

LSTM model - Main block
Epoch 1/25, Loss: 7.4517
Epoch 2/25, Loss: 5.3762
Epoch 3/25, Loss: 4.0271
Epoch 4/25, Loss: 3.7364
Epoch 5/25, Loss: 3.1373
Epoch 6/25, Loss: 2.8577
Epoch 7/25, Loss: 2.5727
Epoch 8/25, Loss: 2.4156
Epoch 9/25, Loss: 2.2617
Epoch 10/25, Loss: 2.1573
Epoch 11/25, Loss: 2.0718
Epoch 12/25, Loss: 2.0088
Epoch 13/25, Loss: 1.9885
Epoch 14/25, Loss: 1.9623
Epoch 15/25, Loss: 1.9314
Epoch 16/25, Loss: 1.8767


In [ ]:
"""
# Step5.2: Prepare fine-tuned model by loading model weights from ckpt file
checkpoint_path = "/content/multirun/2025-02-11/15-40-14/0/checkpoints/epoch=13-step=1400.ckpt"

# Load the model
model = MoiraiForecast.load_from_checkpoint(checkpoint_path,
      prediction_length=PREDICTION_LENGTH,
      context_length=CONTEXT_LENGTH,
      patch_size=PATCH_SIZE,
      num_samples=100,
      target_dim=1,
      feat_dynamic_real_dim=dataset.num_feat_dynamic_real,
      past_feat_dynamic_real_dim=dataset.num_past_feat_dynamic_real)
"""

In [ ]:
# Fine-tuning in regards to https://github.com/SalesforceAIResearch/uni2ts/blob/main/README.md#fine-tuning
# Step 1 Set Data Path Directory
!echo "CUSTOM_DATA_PATH=/content/uni2ts/" >> .env

In [ ]:
!echo "PYTHONPATH=/content/uni2ts" >> .env

In [ ]:
!cat .env

CUSTOM_DATA_PATH=/content/uni2ts/
PYTHONPATH=/content/uni2ts


In [ ]:
# Step 2.1 process dataset
!python -m uni2ts.data.builder.simple customdataset /content/uni2ts/230401_250108_PT1H_Solcast_reduced_features.csv --dataset_type wide_multivariate

Inferred frequency: H. Using this value for the 'freq' parameter.
Generating train split: 1 examples [00:00,  4.58 examples/s]
Saving the dataset (1/1 shards): 100% 1/1 [00:00<00:00, 263.00 examples/s]


In [ ]:
# Step 2.2 Set offset
!python -m uni2ts.data.builder.simple customdataset /content/uni2ts/230401_250108_PT1H_Solcast_reduced_features.csv --date_offset '2024-08-31 07:00:00'

                     dc_power  air_temp  azimuth  cloud_opacity  dhi  dni  ghi  zenith
date                                                                                  
2023-04-01 01:00:00       0.0         8      -27           37.4    0    0    0     125
2023-04-01 02:00:00       0.0         8      -43           46.6    0    0    0     120
2023-04-01 03:00:00       0.0         8      -57           40.6    0    0    0     112
2023-04-01 04:00:00       0.0         7      -70           47.6    0    0    0     103
2023-04-01 05:00:00       0.0         7      -81           58.2    0    0    0      93
...                       ...       ...      ...            ...  ...  ...  ...     ...
2024-08-31 03:00:00       0.0        19      -55            0.0    0    0    0     107
2024-08-31 04:00:00       0.0        19      -67           16.5    0    0    0      98
2024-08-31 05:00:00       0.0        19      -79           39.8    7    0    7      88
2024-08-31 06:00:00       0.0        19    

In [ ]:
#!mv content/uni2ts/cli content/uni2ts/src/uni2ts/

In [ ]:
# Step 3 (move cli dir to src/uni2ts first!!!!)
# Set Batch size here /content/uni2ts/src/uni2ts/cli/conf/finetune/default.yaml to lower value
# For moirai large with A100 40GB RAM use 16 as batch size in val_dataloader and train_dataloader section

#!python -m uni2ts.cli.train -cp conf/finetune run_name=example_run model=moirai_1.1_R_large data=etth1 val_data=etth1

In [ ]:
# Use Hydra's Sweeping Feature for Hyperparameter Search
!python -m uni2ts.cli.train --multirun -cp conf/finetune run_name=example_run model=moirai_1.1_R_base data=customdataset val_data=customdataset \
  model.module_kwargs.dropout_p=0.2 \
  trainer.max_epochs=25 \
  model.lr=1e-7 \
  train_dataloader.batch_size=24 \
  val_dataloader.batch_size=24

[2025-02-11 15:40:14,959][HYDRA] Launching 1 jobs locally
[2025-02-11 15:40:14,959][HYDRA] 	#0 : run_name=example_run model=moirai_1.1_R_base data=customdataset val_data=customdataset model.module_kwargs.dropout_p=0.2 trainer.max_epochs=25 model.lr=1e-07 train_dataloader.batch_size=24 val_dataloader.batch_size=24
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
[2025-02-11 15:40:16,903][datasets][INFO] - PyTorch version 2.4.1 available.
[2025-02-11 15:40:16,903][datasets][INFO] - TensorFlow version 2.18.0 available.
[2025-02-11 15:40:16,904][datasets][INFO] - JAX version 0.4.33 available.
Seed set to 0
2025-02-11 15:40:17.397359: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739288417.422296   43645 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register 

In [ ]:
# Show Tensorboard with results
#%load_ext tensorboard
#%tensorboard --logdir /content/multirun


In [ ]:
#Export outputs
#!zip -r outputs.zip /content/outputs

In [ ]:
# Evaluation
#!python -m uni2ts.cli.eval run_name=example_eval_1 model=moirai_1.0_R_small model.patch_size=32 model.context_length=1000 data=etth1_val

In [ ]:
#import gc

# Invoke garbage collector
#gc.collect()

# Clear GPU cache
#torch.cuda.empty_cache()